In [1]:
from cvrp_parser import parse_cvrplib_uchoa_2014

In [2]:
# read the file Uchoa_et_al_2014/X-n101-k25.vrp
file_path = 'Uchoa_et_al_2014/X-n670-k130.vrp'
#load the text into a string
with open(file_path, 'r') as file:
    vrp_text = file.read()
# parse the CVRP data
inst  = parse_cvrplib_uchoa_2014(vrp_text)

In [3]:
print(inst.name, inst.n_customers, inst.capacity, inst.edge_weight_type)
print("Depot coords:", inst.coords[0])
print("First customer coords:", inst.coords[1])
print("Distance(0,1):", inst.dist(0,1))

X-n670-k130 669 129 EUC_2D
Depot coords: [612. 199.]
First customer coords: [864. 279.]
Distance(0,1): 264.39364591457183


In [4]:
from constructors import clarke_wright_parallel
from utils import build_distance_matrix, is_feasible
from plot_utils import save_solution, plot_solution_from_file

# Clarke–Wright
cw = clarke_wright_parallel(inst.coords, inst.demand, inst.capacity, inst.edge_weight_type)
print("[CW] routes:", len(cw.routes), "cost:", cw.cost,
      "feasible:", is_feasible(cw.routes, inst.demand, inst.capacity, must_cover_all=True, n_customers=inst.n_customers))

# 1) Save to .sol
sol_path = save_solution(inst, cw.routes, cw.cost)  # -> solutions/X-n101-k25/X-n101-k25.sol

# 2) Plot from .sol
png_path = plot_solution_from_file(inst, sol_path)    # -> solutions/X-n101-k25/X-n101-k25.png
print("Wrote:", sol_path, "and", png_path)

[CW] routes: 147 cost: 158715.27300808093 feasible: True
Wrote: solutions\X-n670-k130\X-n670-k130.sol and solutions\X-n670-k130\X-n670-k130.png


In [5]:
from local_search import local_search
# Local search
# Use verbose=True for quick logging, or pass your own logger
ls = local_search(
    coords=inst.coords,
    demand=inst.demand,
    Q=inst.capacity,
    routes=[r[:] for r in cw.routes],
    edge_weight_type=inst.edge_weight_type,
    verbose=True,
    time_limit_sec=60.0,   # stop early, keep best
    improvement_eps=1e-4
)

print(f"BEST cost: {ls.cost:.2f}, routes: {len(ls.routes)}")
#write the LS solution
sol_path_ls = save_solution(inst, ls.routes, ls.cost)
png_path_ls = plot_solution_from_file(inst, sol_path_ls)
print("Wrote:", sol_path_ls, "and", png_path_ls)

18:54:36 [INFO] LS start | routes=147 cost=158715.2730 | time_limit=60.0s | no_gain_passes=2 | impr_eps=0.0001
18:54:36 [INFO]   -> new BEST after 2-opt r=17 i=2 k=5 | best=158712.3730
18:54:36 [INFO] [2-opt] r=17 i=2 k=5 kept
18:54:36 [INFO]   -> new BEST after 2-opt r=17 i=3 k=4 | best=158712.1978
18:54:36 [INFO] [2-opt] r=17 i=3 k=4 kept
18:54:36 [INFO]   -> new BEST after 2-opt r=17 i=3 k=5 | best=158696.8454
18:54:36 [INFO] [2-opt] r=17 i=3 k=5 kept
18:54:36 [INFO]   -> new BEST after 2-opt r=55 i=1 k=2 | best=158675.2049
18:54:36 [INFO] [2-opt] r=55 i=1 k=2 kept
18:54:36 [INFO]   -> new BEST after 2-opt r=83 i=4 k=5 | best=158671.8513
18:54:36 [INFO] [2-opt] r=83 i=4 k=5 kept
18:54:36 [INFO]   -> new BEST after 2-opt r=83 i=3 k=4 | best=158655.2915
18:54:36 [INFO] [2-opt] r=83 i=3 k=4 kept
18:54:36 [INFO]   -> new BEST after 2-opt r=88 i=1 k=4 | best=158650.6898
18:54:36 [INFO] [2-opt] r=88 i=1 k=4 kept
18:54:36 [INFO]   -> new BEST after 2-opt r=88 i=2 k=4 | best=158644.6765
18:

BEST cost: 158464.40, routes: 147
Wrote: solutions\X-n670-k130\X-n670-k130.sol and solutions\X-n670-k130\X-n670-k130.png


In [ ]:
from ils import run_ils, ILSConfig
cfg = ILSConfig(
    time_limit_sec=70.0,
    ls_time_slice_sec=2.0,
    max_passes_without_gain=2,
    k_nearest=20,
    T_init_factor=0.01,
    T_cooling=0.98,
    seed=0,
    verbose=True,
)
ils_out = run_ils(
    coords=inst.coords,
    demand=inst.demand,
    Q=inst.capacity,
    routes_initial=[r[:] for r in cw.routes],
    edge_weight_type=inst.edge_weight_type,
    round_euclidean=False,
    cfg=cfg,
)

print(f"[ILS] best_cost={ils_out.cost:.2f} routes={len(ils_out.routes)} in {ils_out.elapsed_sec:.1f}s (iters={ils_out.iterations})")

# 3) Check, save, plot
ok = is_feasible(ils_out.routes, inst.demand, inst.capacity, must_cover_all=True, n_customers=inst.n_customers)
print("Feasible:", ok)

sol_path = save_solution(inst, ils_out.routes, ils_out.cost)
png_path = plot_solution_from_file(inst, sol_path)
print("Wrote:", sol_path, "and", png_path)

In [ ]:
from tabu_search import tabu_search

ts = tabu_search(
    coords=inst.coords,
    demand=inst.demand,
    Q=inst.capacity,
    routes_init=[r[:] for r in cw.routes],
    edge_weight_type=inst.edge_weight_type,
    time_limit_sec=70.0,             # hard wall
    max_iters=100000,
    max_no_improve=500,              # patience
    tabu_tenure=15,                  # or tabu_tenure_range=(12,20)
    aspiration=True,
    k_nearest=20,
    max_intra_candidates_per_route=64,
    max_inter_candidates_per_route=128,
    verbose=True,
    seed=42,
)

print(f"[TS] routes={len(ts.routes)} cost={ts.cost:.2f}")

# Save & plot
sol_path = save_solution(inst, ts.routes, ts.cost)
png_path = plot_solution_from_file(inst, sol_path)
print("Wrote:", sol_path, "and", png_path)

In [8]:
from alns import alns

alns_sol = alns(
  coords=inst.coords, demand=inst.demand, Q=inst.capacity,
  routes_init=[r[:] for r in cw.routes],
  edge_weight_type=inst.edge_weight_type,
  # intensity & acceptance
  remove_fraction=(0.08, 0.18),
  start_temperature=500.0, end_temperature=1.0, cooling_rate=0.996,
  use_destroy=("shaw","worst","clustered_routes"),
  use_repair=("regret3","regret2"),
  reaction=0.3, segment_length=40,
  use_local_search=True, ls_time_limit_sec=1.2, ls_max_passes_without_gain=2,
  time_limit_sec=120, max_iters=300000,
  verbose=True, log_every=10, seed=44,
)


print(f"[ALNS] routes={len(alns_sol.routes)} cost={alns_sol.cost:.2f}")
sol_path = save_solution(inst, alns_sol.routes, alns_sol.cost)
png_path = plot_solution_from_file(inst, sol_path)
print("Wrote:", sol_path, "and", png_path)

18:54:58 [INFO] ALNS start | routes=147 cost=158715.27 | remove=[53,120] | ops D=['worst', 'shaw', 'clustered_routes'], R=['regret2', 'regret3'] | seed=44
18:55:44 [INFO] [ALNS] it=10 T=480.36 Δ=+1687.01 curr=158557.24 best=158467.88 D=clustered_routes R=regret3 q=78 routes=147
18:56:15 [INFO] [ALNS] it=20 T=461.48 Δ=+1684.97 curr=158499.63 best=158467.88 D=worst R=regret2 q=60 routes=147
18:56:50 [INFO] [ALNS] it=30 T=443.35 Δ=+4062.02 curr=158262.52 best=158262.52 D=clustered_routes R=regret3 q=111 routes=147


[ALNS] routes=147 cost=158255.58
Wrote: solutions\X-n670-k130\X-n670-k130.sol and solutions\X-n670-k130\X-n670-k130.png


In [ ]:
from lns import lns

lns_sol = lns(
    coords=inst.coords, demand=inst.demand, Q=inst.capacity,
    routes_init=[r[:] for r in cw.routes],
    edge_weight_type=inst.edge_weight_type,
    # knobs
    remove_fraction=(0.12, 0.25),
    use_sa=True, start_temperature=900.0, end_temperature=5.0, cooling_rate=0.996,
    # intensification
    use_local_search=True, ls_time_limit_sec=1.0, ls_max_passes_without_gain=1,
    # budget/logging
    time_limit_sec=90, max_iters=200000,
    verbose=True, log_every=5, log_detail=True,
    seed=7,
)

print(f"[LNS] routes={len(lns_sol.routes)} cost={lns_sol.cost:.2f}")
sol_path = save_solution(inst, lns_sol.routes, lns_sol.cost)
png_path = plot_solution_from_file(inst, sol_path)
print("Wrote:", sol_path, "and", png_path)
Tuning tips
Destruction: Increase remove_fraction (e.g., (0.15, 0.30)) to break structure harder on big cases.

Acceptance: If progress is slow, raise start_temperature o